In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Decision Parameters

In [ ]:
# loss previously obtained when the loop stoped
min_ant =   100  #upload here the values of the loss on which your training stopped 
min_ant_v =  100 ##upload here the values of the loss on which your training stopped

In [ ]:
### Network 
epochs      = 200
batch_size = 32
BATCH_SIZE =32
temperat = 0.1

# Path where the models will be saved 
filepath_save_weights_v = '/content/drive/MyDrive/Tese/fold2_simclr_hflip_crop_rot_sem_imagw_prbem_v'
filepath_save_weights = '/content/drive/MyDrive/Tese/fold2_simclr_hflip_crop_rot_sem_imagw_prbem'
filepath_checkpoint = '/content/drive/MyDrive/Tese/fold2_simclr_hflip_crop_rot_sem_imagw_prbem-aqui'

# if is Second run = true
second = False
NUM_TRAIN_SAMPLES = 17731
NUM_VAL_SAMPLES = 7600
BATCH_SIZE = 32

#----------------------------------
## LR
decay_steps =1500  # 2 epochs  # 5540/554=10 epochs decresce  
initial_learning_rate=0.0001
decay_r=0.96
stair=False
#----------------------------------
## Show validation loss
validation = True
# validation = False

#-----------------------------------
## Platform weights and biases
wgt = False
# wgt = True

#-----------------------------------
## Resnet50
weight_resnet = True #true =imagenet
res = 'new'

## Resnet50
rede = 'resnet50' 
h1 = 256
h2 = 128


#----------------------------------
### Dataset choice
dados = 'flow'
# dados = 'tfdataset'


#----------------------------------
### Data Augmentation
# Random flips
flip_left = True
flip_up = False 

# Random Rotations
rot90 = True

# Random Crop
# ccrop = True
ccrop = True

# Randomly apply transformation (color distortions) with probability p.
color_j = False
str_j = 0.25
color_d = False 

# Random Gaussian Blur
gaussian_b = False 

#Network Parameters
image_size  = 224
input_shape = (image_size, image_size, 3)
test_batch_size  = 1

kernel_size = 3
filters     = 16
latent_dim  = 128

img_height = 224
img_width = 224
IMG_SHAPE = 224

In [ ]:
if second == False:
  !unzip /content/drive/MyDrive/Tese/ISIC_2019_pre-processed_cs_final.zip -d /content

<a></a>
## 1.Imports

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

# import tensorflow_datasets as tfds
import os 
# from simclr_preproce import * 

from imutils import paths
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import matplotlib.pyplot as plt
from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import cv2

from sklearn.preprocessing import LabelEncoder

import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

# import tensorflow_datasets as tfds
import os 
# from simclr_preproce import * 

from imutils import paths
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# from wandb.keras impimport csv
import pandas as pd
import numpy as np
import os,sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from collections import Counter

sys.path.append('/content/drive/MyDrive/Tese/')

from visuals import plot_grouped_2bars
import math
import shutil
from sklearn.utils import shuffle

import glob

import os, os.path
from ast import literal_eval
from make_dir_versao_writedisco import *

import cv2

from PIL import Image


import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Convolution2D, Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras import layers, losses

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
#from tensorflow.python.keras.preprocessing.image import image_dataset_from_directory

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

# from mpl_toolkits.mplot3d import Axes3Dort WandbCallback
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import random

from tensorflow.keras.callbacks import LearningRateScheduler
# from wandb.keras import WandbCallback

# Random seed fixation
tf.random.set_seed(666)
np.random.seed(666)

!nvidia-smi

if wgt ==  True:
    # Install wandb for experiment tracking
    !pip install --upgrade https://github.com/wandb/client/archive/feature/code-save.zip
    import wandb
    wandb.login()

# Random seed fixation
tf.random.set_seed(666)
np.random.seed(666)

Dataset gathering and preparation

In [ ]:
def gaussian_blur(image, kernel_size, sigma, padding='SAME'):
      """Blurs the given image with separable convolution.
      Args:
        image: Tensor of shape [height, width, channels] and dtype float to blur.
        kernel_size: Integer Tensor for the size of the blur kernel. This is should
          be an odd number. If it is an even number, the actual kernel size will be
          size + 1.
        sigma: Sigma value for gaussian operator.
        padding: Padding to use for the convolution. Typically 'SAME' or 'VALID'.
      Returns:
        A Tensor representing the blurred image.
      """
      radius = tf.cast(kernel_size / 2, dtype=tf.int32)
      kernel_size = radius * 2 + 1
      x = tf.cast(tf.range(-radius, radius + 1), dtype=tf.float32)
      blur_filter = tf.exp(-tf.pow(x, 2.0) /
                          (2.0 * tf.pow(tf.cast(sigma, dtype=tf.float32), 2.0)))
      blur_filter /= tf.reduce_sum(blur_filter)
      # One vertical and one horizontal filter.
      blur_v = tf.reshape(blur_filter, [kernel_size, 1, 1, 1])
      blur_h = tf.reshape(blur_filter, [1, kernel_size, 1, 1])
      num_channels = tf.shape(image)[-1]
      blur_h = tf.tile(blur_h, [1, 1, num_channels, 1])
      blur_v = tf.tile(blur_v, [1, 1, num_channels, 1])
      expand_batch_dim = image.shape.ndims == 3
      if expand_batch_dim:
        # Tensorflow requires batched input to convolutions, which we can fake with
        # an extra dimension.
        image = tf.expand_dims(image, axis=0)
      blurred = tf.nn.depthwise_conv2d(
          image, blur_h, strides=[1, 1, 1, 1], padding=padding)
      blurred = tf.nn.depthwise_conv2d(
          blurred, blur_v, strides=[1, 1, 1, 1], padding=padding)
      if expand_batch_dim:
        blurred = tf.squeeze(blurred, axis=0)
      return blurred

def random_apply(func, p, x):
  """Randomly apply function func to x with probability p."""
  return tf.cond(
      tf.less(
          tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
          tf.cast(p, tf.float32)), lambda: func(x), lambda: x)

class CustomAugment(object):
    def __call__(self, sample):

        # Random flips
        if flip_left == True:
          # print('Flip left right activated')
          sample = self._random_apply(tf.image.flip_left_right, sample, p=0.5)
        if flip_up == True:
          print('Flip up down activated')
          sample = self._random_apply(tf.image.flip_up_down,sample,p=0.5)
      
        # Random Rotations
        if rot90 == True:
          # print('Rot activated')
          sample = self._random_apply(self._rotate_image, sample, p=1)
      
        # Random Crop
        if ccrop == True:
          # print('Crop activated')
          sample = self._random_apply(self._center_crop, sample, p=0.5)
        
        # Randomly apply transformation (color distortions) with probability p.
        if color_j == True:
          # print('color_jitter activated')
          sample = self._random_apply(self._color_jitter, sample, p=0.5)
        if color_d == True:
          # print('color_drop activated')
          sample = self._random_apply(self._color_drop, sample, p=0.2)

        # Random Gaussian Blur
        if gaussian_b == True:
          print('gaussian_b activated')
          sample = self._random_apply(self._random_blur, sample, p = 0.2)

        return sample

    def _color_jitter(self, x, s=str_j):
        # one can also shuffle the order of following augmentations
        # each time they are applied.
        x = tf.image.random_brightness(x, max_delta=0.8*s)
        x = tf.image.random_contrast(x, lower=1-0.8*s, upper=1+0.8*s)
        x = tf.image.random_saturation(x, lower=1-0.8*s, upper=1+0.8*s)
        x = tf.image.random_hue(x, max_delta=0.2*s)
        x = tf.clip_by_value(x, 0, 1)
        return x
    
    def _color_drop(self, x):
        x = tf.image.rgb_to_grayscale(x)
        x = tf.tile(x, [1, 1, 1, 3])
        return x

    def _rotate_image(self, x):
        return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)) # Rotate 0, 90, 180, 270 degrees


    def _center_crop(self,x):
        num = random.choice([0.8,0.9])
        # print(num)
        x = tf.image.central_crop(x, num)
        x = tf.image.resize(x, [224, 224], method='lanczos5',preserve_aspect_ratio=True)
        return x

    def _random_blur(self,x):
        """Randomly blur an image.
        Args:
          image: `Tensor` representing an image of arbitrary size.
          height: Height of output image.
          width: Width of output image.
          p: probability of applying this transformation.
        Returns:
          A preprocessed image `Tensor`.
        """
        height = 224
        width = 224
        del width
        def _transform(x):
          sigma = tf.random.uniform([], 0.1, 2.0, dtype=tf.float32)
          return gaussian_blur(
              x, kernel_size=height//10, sigma=sigma, padding='SAME')
        return random_apply(_transform, p=1.0, x=x)



    def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)
    
# Build the augmentation pipeline
data_augmentation = Sequential([Lambda(CustomAugment())])


<a></a>
## 2. Online Data Augmentation

In [ ]:
from tensorflow.python.keras.applications.resnet import ResNet50, preprocess_input

if dados == 'flow':
    train_dataGen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        # rescale=1./255
                                       ) 

    seed = 1
    train_ds = train_dataGen.flow_from_directory(
              directory = '/content/ISIC_2019_pre-processed_cs_final/train',
              class_mode = None,
              color_mode = 'rgb',
              batch_size = batch_size,
              target_size = (image_size,image_size),
              shuffle = True,
              seed = seed
        )
    
    valid_dataGen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        # rescale=1./255
                                       ) 

    path = "/content/ISIC_2019_pre-processed_cs_final/train/train_ground_class.csv"
    gt_train = pd.read_csv(path)

    path = "/content/ISIC_2019_pre-processed_cs_final/valid/val_ground_class.csv"
    gt_val = pd.read_csv(path)

    seed = 2
    valid_ds = valid_dataGen.flow_from_directory(
              directory = '/content/ISIC_2019_pre-processed_cs_final/valid',
              class_mode = None,
              color_mode = 'rgb',
              batch_size = batch_size,
              target_size = (image_size,image_size),
              shuffle = True,
              seed = seed
        )


Found 17731 images belonging to 8 classes.
Found 7600 images belonging to 8 classes.


<a></a>
## 3.Build ResNet Architecture


In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
if weight_resnet == True:
  MODEL = ResNet50(include_top=False, weights= 'imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)
  # MODEL.summary()
  print('imagenet weighths loaded')
elif weight_resnet == False:
  MODEL = ResNet50(include_top=False, weights= None, input_tensor=None, input_shape=(224,224,3), pooling=None)
  # MODEL.summary()

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
MODEL.trainable = True
inputs  = Input((224, 224, 3))
h = MODEL(inputs, training=False)
h = GlobalAveragePooling2D()(h)
projection_11 = Dense(256,name='Dense_1')(h)
projection_1 = Dropout(0.5)(projection_11)
projection_1 = Activation("relu")(projection_1)
# projection_22 = BatchNormalization()(projection_1)
projection_2 = Dense(128,name='Dense_2')(projection_1)

model = Model(inputs, projection_2)

model.summary()



class_weight_dict = {0: NUM_TRAIN_SAMPLES/3165,
                1: NUM_TRAIN_SAMPLES/9012,
                2: NUM_TRAIN_SAMPLES/2326,
                3: NUM_TRAIN_SAMPLES/607,
                4: NUM_TRAIN_SAMPLES/1837,
                5: NUM_TRAIN_SAMPLES/167,
                6: NUM_TRAIN_SAMPLES/177,
                7: NUM_TRAIN_SAMPLES/440}



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
Dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
Dense_2 (Dense)              (None, 128)               3289

In [ ]:

###---------------------------------------------------------------
# helpers:
def get_negative_mask(batch_size):
    # return a mask that removes the similarity score of equal/similar images.
    # this function ensures that only distinct pair of images get their similarity scores
    # passed as negative examples
    negative_mask = np.ones((batch_size, 2 * batch_size), dtype=bool)
    for i in range(batch_size):
        negative_mask[i, i] = 0
        negative_mask[i, i + batch_size] = 0
    return tf.constant(negative_mask)

# Losses:
cosine_sim_1d = tf.keras.losses.CosineSimilarity(axis=1, reduction=tf.keras.losses.Reduction.NONE)
cosine_sim_2d = tf.keras.losses.CosineSimilarity(axis=2, reduction=tf.keras.losses.Reduction.NONE)


def _cosine_simililarity_dim1(x, y):
    v = cosine_sim_1d(x, y)
    return v


def _cosine_simililarity_dim2(x, y):
    # x shape: (N, 1, C)
    # y shape: (1, 2N, C)
    # v shape: (N, 2N)
    v = cosine_sim_2d(tf.expand_dims(x, 1), tf.expand_dims(y, 0))
    return v


def sim_func_dim1(x, y):
    # x shape: (N, 1, C)
    # y shape: (N, C, 1)
    # v shape: (N, 1, 1)
    v = tf.matmul(tf.expand_dims(x, 1), tf.expand_dims(y, 2))
    return v


def sim_func_dim2(x, y):
    v = tf.tensordot(tf.expand_dims(x, 1), tf.expand_dims(tf.transpose(y), 0), axes=2)
    # x shape: (N, 1, C)
    # y shape: (1, C, 2N)
    # v shape: (N, 2N)
    return v
###---------------------------------------------------------------

def plot_images(imgs, imgs_pred):
    fig=plt.figure(figsize=(10, 20))
    columns = 2
    rows    = 8

    for i in range(1, 9, 2):
        fig.add_subplot(rows, columns, i)
        plt.imshow(imgs[i])
        
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(imgs_pred[i])
    plt.show()



###---------------------------------------------------------------

@tf.function
def train_step(xis, xjs, model, optimizer, criterion, temperature):
    with tf.GradientTape() as tape:
        # print(xis)
        # tf.print(xis)
        
        # xis = tf.keras.applications.resnet.preprocess_input(xis)
        # xis = tf.keras.applications.resnet50.preprocess_input(xis)
        # tf.print(xis)
        # print('prev\n\n  newwww \n\n\n\n\n')
        # # print(xis)
        # xjs = tf.keras.applications.resnet.preprocess_input(xjs) 
        
        zis = model(xis)
        zjs = model(xjs)

        # normalize projection feature vectors
        zis = tf.math.l2_normalize(zis, axis=1)
        zjs = tf.math.l2_normalize(zjs, axis=1) #32,128

        # Cosine Similarity of positive pairs - l_pos
        l_pos = sim_func_dim1(zis, zjs)
        l_pos = tf.reshape(l_pos, (BATCH_SIZE, 1))
        l_pos /= temperature #32,1

        # vector w/ all pairs
        negatives = tf.concat([zjs, zis], axis=0) #64,128

        loss = 0
        
        #1st zis then zjs
        for positives in [zis, zjs]:
            
            # cosine similarity de k=1 ate 2n
            l_neg = sim_func_dim2(positives, negatives) #32,64

            # vector of 0s 
            labels = tf.zeros(BATCH_SIZE, dtype=tf.int32) #32,1

            # Cosine similarity of negative pairs 
            l_neg = tf.boolean_mask(l_neg, negative_mask) #1984 =32*64-64 (64 diagonal of 32*64 matrix)
            l_neg = tf.reshape(l_neg, (BATCH_SIZE, -1)) #1984/32=>  #32,62
            l_neg /= temperature

            # Vector that has the format for sparce cross entropy loss
            logits = tf.concat([l_pos, l_neg], axis=1) 
            # Applies cross entropy loss and compares to loss 0
            loss += criterion(y_pred=logits, y_true=labels)

        #  loss over all the pairs in the batch of size N=2 and take an average
        loss = loss / (2 * BATCH_SIZE)

    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    return loss

@tf.function
def val_step(xis, xjs, model, optimizer, criterion, temperature):
    # with tf.GradientTape() as tape:
    # print(xis)
    # tf.print(xis)
    
    # xis = tf.keras.applications.resnet.preprocess_input(xis)
    # xis = tf.keras.applications.resnet50.preprocess_input(xis)
    # tf.print(xis)
    # print('prev\n\n  newwww \n\n\n\n\n')
    # # print(xis)
    # xjs = tf.keras.applications.resnet.preprocess_input(xjs) 
    
    zis = model(xis)
    zjs = model(xjs)

    # normalize projection feature vectors
    zis = tf.math.l2_normalize(zis, axis=1)
    zjs = tf.math.l2_normalize(zjs, axis=1) #32,128

    # Cosine Similarity of positive pairs - l_pos
    l_pos = sim_func_dim1(zis, zjs)
    l_pos = tf.reshape(l_pos, (BATCH_SIZE, 1))
    l_pos /= temperature #32,1

    # vector w/ all pairs
    negatives = tf.concat([zjs, zis], axis=0) #64,128

    loss = 0
    
    #1st zis then zjs
    for positives in [zis, zjs]:
        
        # cosine similarity de k=1 ate 2n
        l_neg = sim_func_dim2(positives, negatives) #32,64

        # vector of 0s 
        labels = tf.zeros(BATCH_SIZE, dtype=tf.int32) #32,1

        # Cosine similarity of negative pairs 
        l_neg = tf.boolean_mask(l_neg, negative_mask) #1984 =32*64-64 (64 diagonal of 32*64 matrix)
        l_neg = tf.reshape(l_neg, (BATCH_SIZE, -1)) #1984/32=>  #32,62
        l_neg /= temperature

        # Vector that has the format for sparce cross entropy loss
        logits = tf.concat([l_pos, l_neg], axis=1) 
        # Applies cross entropy loss and compares to loss 0
        loss += criterion(y_pred=logits, y_true=labels)

    #  loss over all the pairs in the batch of size N=2 and take an average
    loss = loss / (2 * BATCH_SIZE)

    # gradients = tape.gradient(loss, model.trainable_weights)
    # optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    return loss

In [ ]:
# Mask to remove positive examples from the batch of negative samples
# The diagonals are False 
negative_mask = get_negative_mask(BATCH_SIZE)

print('ISIC TRAIN')
if wgt == True: 
  wandb.init(project="_validation_simclr_eval_fcc_rot_imagenet")	

#------------------------- Validation == False ---------------------------------------------------------------------
if dados == 'flow' and validation == False:
    def train_simclr(manager,BATCH_SIZE,model, dataset, optimizer, criterion,
                    temperature=0.1, epochs=100):
        step_wise_loss = []
        epoch_wise_loss = []

        ###
        ckpt.restore(manager.latest_checkpoint)
        if manager.latest_checkpoint:
          print("Restored from {}".format(manager.latest_checkpoint))
        else:
          print("Initializing from scratch.")
        ###

        for epoch in tqdm(range(epochs)):
            for batch in range(17731//BATCH_SIZE):
                  image_batch = next(dataset)
                  if len(image_batch) == BATCH_SIZE:
                      a = data_augmentation(image_batch)
                      b = data_augmentation(image_batch)
                      
                      # Plot original imagem and predicted image from the autoencoder.
                      # print('\n\nPloting reconstructed images')
                      # print('orig, a:')
                      # plot_images(image_batch, a)
                      # print('orig, b:')
                      # plot_images(image_batch, b)

                      loss = train_step(a, b, model, optimizer, criterion, temperature)

                      # print('Train_step completed')
                      step_wise_loss.append(loss)

                      ###
                      ckpt.step.assign_add(1)
                      ###
                     
            epoch_wise_loss.append(np.mean(step_wise_loss))
            
            if wgt == True: 
                  wandb.log({"nt_xentloss": np.mean(step_wise_loss)})
            
            ckpt.epoch.assign_add(1)
            save_path = manager.save()
            print("\nSaved checkpoint for step {}, stored_epoch {}, loop_epoch {}".format(int(ckpt.step), int(ckpt.epoch), epoch + 1))
            print("loss {:1.6f}".format(np.mean(step_wise_loss)))

            if np.mean(step_wise_loss) <= min_ant:
              if np.mean(step_wise_loss) <= min(epoch_wise_loss):
                  #saves the best weights with lowest loss
                  print(' -----> Best loss so far {:1.6f} <-----'.format(np.mean(step_wise_loss)))
                  model.save_weights(filepath_save_weights)
                       
            # print("stored_epoch: {}  loss: {:.6f} loss_val: {:.6f} ".format( int(ckpt.epoch), np.mean(step_wise_loss),np.mean(step_wise_loss_v)))

        return epoch_wise_loss, model 



#------------------------- Validation == True ---------------------------------------------------------------------
elif dados == 'flow' and validation == True:
    def train_simclr(manager,BATCH_SIZE,model, dataset, dataset_v, optimizer, criterion,
                    temperature=0.1, epochs=100):
        step_wise_loss = []
        epoch_wise_loss = []
        step_wise_loss_v = []
        epoch_wise_loss_v = []

        ###
        ckpt.restore(manager.latest_checkpoint)
        if manager.latest_checkpoint:
          print("Restored from {}".format(manager.latest_checkpoint))
        else:
          print("Initializing from scratch.")
        ###

        for epoch in tqdm(range(epochs)):
            for batch in range(17731//BATCH_SIZE):
                  image_batch = next(dataset)
                  image_batch_v = next(dataset_v)
                  if len(image_batch) == BATCH_SIZE:
                      a = data_augmentation(image_batch)
                      b = data_augmentation(image_batch)
                      loss = train_step(a, b, model, optimizer, criterion, temperature)
                      step_wise_loss.append(loss)
                      ###
                      ckpt.step.assign_add(1)
                      ###

                  if len(image_batch_v) == BATCH_SIZE:     
                      a_v = data_augmentation(image_batch_v)
                      b_v = data_augmentation(image_batch_v)
                      loss_v = val_step(a_v, b_v, model, optimizer, criterion, temperature)
                      step_wise_loss_v.append(loss_v)
                     
            epoch_wise_loss.append(np.mean(step_wise_loss))
            epoch_wise_loss_v.append(np.mean(step_wise_loss_v))
       
            if wgt == True: 
                wandb.log({"nt_xentloss": np.mean(step_wise_loss)})
                wandb.log({"nt_xentloss_v": np.mean(step_wise_loss_v)})
                
            ckpt.epoch.assign_add(1)
            save_path = manager.save()
            print("\nSaved checkpoint for step {}, stored_epoch {}, loop_epoch {}".format(int(ckpt.step), int(ckpt.epoch), epoch + 1))
            print("loss {:1.6f},  loss_val: {:.6f}".format(np.mean(step_wise_loss),np.mean(step_wise_loss_v)))

            if np.mean(step_wise_loss) <= min_ant:
              if np.mean(step_wise_loss) <= min(epoch_wise_loss):
                  #saves the best weights with lowest loss
                  print(' -----> Best loss so far train loss: {:1.6f}  val_loss: {:1.6f}<-----'.format(np.mean(step_wise_loss),np.mean(step_wise_loss_v)))
                  model.save_weights(filepath_save_weights)
            if np.mean(step_wise_loss_v) <= min_ant_v:
              if np.mean(step_wise_loss_v) <= min(epoch_wise_loss_v):
                  #saves the best weights with lowest loss
                  print(' -----> Best loss so far validation loss: {:1.6f}  val_loss: {:1.6f}<-----'.format(np.mean(step_wise_loss),np.mean(step_wise_loss_v)))
                  model.save_weights(filepath_save_weights_v)
                       
            # print("stored_epoch: {}  loss: {:.6f} loss_val: {:.6f} ".format( int(ckpt.epoch), np.mean(step_wise_loss),np.mean(step_wise_loss_v)))
            
        return epoch_wise_loss, model  

#############################
## Training
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                          reduction=tf.keras.losses.Reduction.SUM)
##################


learning_rate_fn = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps, decay_rate=decay_r, staircase=stair) # em escada 
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)

###################

# decay_steps = 1000
# lr_decayed_fn = tf.keras.experimental.CosineDecay(initial_learning_rate=0.1, decay_steps=decay_steps)
# optimizer = tf.keras.optimizers.SGD(lr_decayed_fn)

##################

print(optimizer)

# if rede == 'resnet50':
#   resnet_simclr_2 = get_resnet_simclr(weight_resnet,h1, h2)
# elif rede == 'encoder':
#    resnet_simclr_2 = get_resnet_simclr(weight_resnet,enc_h1, enc_h2)


###
ckpt = tf.train.Checkpoint(step=tf.Variable(1), epoch = tf.Variable(0), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, filepath_checkpoint, max_to_keep=3)
###

print('\n\n Training the model \n')
if validation == False:
    epoch_wise_loss, resnet_simclr  = train_simclr(manager,batch_size,model, train_ds, optimizer, criterion,temperature=temperat, epochs=epochs)
else:
    epoch_wise_loss, resnet_simclr  = train_simclr(manager,batch_size,model, train_ds, valid_ds, optimizer, criterion,temperature=temperat, epochs=epochs)

# resnet_simclr.save_weights('/content/drive/MyDrive/Tese/6-unsper-resnet-61FINAL05dataaug')


plt.style.use("ggplot")
plt.figure()
plt.plot(epoch_wise_loss)
plt.title("tau = 0.1 h1=256 h2=128 ")
plt.show()
